In [1]:
%pip install -e git+https://github.com/UN-GCPDS/python-gcpds.MI_prediction.git#egg=MI_prediction

Obtaining MI_prediction from git+https://github.com/UN-GCPDS/python-gcpds.MI_prediction.git#egg=MI_prediction
  Updating ./src/mi-prediction clone
  Running command git fetch -q --tags
  Running command git reset --hard -q 62ab6dd2cd9933fd260a0c693c3279e98614819e
  Preparing metadata (setup.py) ... done
  Attempting uninstall: MI_prediction
    Found existing installation: MI-prediction 0.1
    Uninstalling MI-prediction-0.1:
      Successfully uninstalled MI-prediction-0.1
  Running setup.py develop for MI_prediction
Note: you may need to restart the kernel to use updated packages.


In [83]:
from MI_prediction.Utils.Datasets import  DataLoader_Rest, get_epochs, get_runs, get_labels
from MI_prediction.Datasets import Cho2017_Rest, BNCI2014001_Rest
from MI_prediction.Utils.Preprocess import filterbank_preprocessor, filterbank, FBCSP, moments, scaler_list
from MI_prediction.Utils.Windowers import create_windows
from MI_prediction.Utils.Datautils import features_by_channels
from MI_prediction.Models import DLSMR
from braindecode.preprocessing.preprocess import exponential_moving_standardize, preprocess, Preprocessor, scale
from scikeras.wrappers import KerasRegressor 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import welch

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split,StratifiedKFold,cross_val_score,GridSearchCV

from sklearn.preprocessing import StandardScaler
from mne.preprocessing import compute_current_source_density
import tensorflow as tf

import time
import pickle

# GIGAScience Cho2017

In [2]:
dl = DataLoader_Rest(dataset_name=Cho2017_Rest())

subjects = np.arange(1,53)
subjects = subjects[(subjects!=32) & (subjects!=46) & (subjects!=49)]

duration=66

## Preprocessing

### For moments

In [4]:
fb = filterbank_preprocessor([(8.,15.),(15.,25.)], method='iir',iir_params={'order':5,'ftype':'butter'})

preprocessors_mts = [
        Preprocessor('pick_types', eeg=True, meg=False, stim=False),  # Keep EEG sensors
        Preprocessor('pick_channels', ch_names=['C3', 'C4', 'Cz', 'P3', 'P4', 'Pz']),  # Keep only EEG channels
        Preprocessor(scale, factor=1e3, apply_on_array=True),  # Convert from V to uV
        Preprocessor('filter',l_freq=1., h_freq=None, method='iir',iir_params={'order':5,'ftype':'butter'})
        ]

Ch_prep = [
        Preprocessor(compute_current_source_density,copy=False,apply_on_array=False)
        ]

st_offsets_mts, ed_offsets_mts = create_windows(win = 2, start_offset = 5, end_offset = -6, mi_duration = duration, overlap = 0.5)

band to filter: (8.0, 15.0) Hz
band to filter: (15.0, 25.0) Hz


### For PSD

In [5]:
preprocessors_psd = [
        Preprocessor('pick_types', eeg=True, meg=False, stim=False),  # Keep EEG sensors
        Preprocessor('pick_channels', ch_names=['C3', 'C4', 'Cz', 'P3', 'P4', 'Pz']), 
        Preprocessor(scale, factor=1e3, apply_on_array=True),  # Convert from V to uV
        Preprocessor('filter',l_freq=8., h_freq=30., method='iir',iir_params={'order':5,'ftype':'butter'}),
        Preprocessor(compute_current_source_density,copy=False,apply_on_array=False)
        ]
st_offsets_psd, ed_offsets_psd = create_windows(win = 55, start_offset = 5, end_offset = -6, mi_duration = duration, overlap = 0)

## Feature extraction

In [6]:
Epochs_mts = []
Epochs_psd = []

for s in subjects:
    print ("Subject: {}".format(s))
    tic = time.time()

    dl.load_data(subject_ids=[s])

    ds_f = filterbank(dl, preprocess=preprocessors_mts, filters=fb, channels_prep=Ch_prep)
    trials = [tr.get_trials(start_offset=st_offsets_mts, end_offset=ed_offsets_mts) for tr in ds_f]

    epochs_mts = []
    for nw in trials[0].keys():
        epochs_mts.append(np.concatenate([np.expand_dims(moments(get_epochs(epoch[nw].split('session')['session_0'])),axis=3) for epoch in trials],axis=-1))

    Epochs_mts.append(np.concatenate(epochs_mts, axis=2))

    fs = dl.dataset.datasets[0].raw.info['sfreq']

    dl.preprocess_data(preprocessors=preprocessors_psd)
    trials_psd = dl.get_trials(start_offset=st_offsets_psd, end_offset=ed_offsets_psd)

    Epochs_psd.append(welch(get_epochs(trials_psd['win_0'])[0],fs=fs,nperseg=int(2*fs))[-1])

    print("Elapsed: {}".format(time.time()-tic))

Epochs_mts = np.concatenate(Epochs_mts, axis=0)
Epochs_psd = np.concatenate(Epochs_psd, axis=0)

save_path = '/Volumes/GoogleDrive/Mi unidad/PhD_Thesis/Data/GIGA/2Class/'

Epochs_rest = {
    'Subjects': subjects,
    'Epochs_mts': Epochs_mts,
    'Epochs_psd': Epochs_psd
    }
pickle.dump(Epochs_rest, open(save_path+'Epochs_rest.pkl', 'wb'))

Subject: 1
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

IIR filter parameters
---------------------
Butterworth highpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 10 (effective, after forward-backward)
- Cutoff at 1.00 Hz: -6.02 dB

Fitted sphere radius:         96.2 mm
Origin head coordinates:      -0.8 14.4 43.1 mm
Origin device coordinates:    -0.8 14.4 43.1 mm
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 15 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 20 (effective, after forward-backward)
- Cutoffs at 8.00, 15.00 Hz: -6.02, -6.02 dB

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

IIR filter parameters
---------------------
Butterworth highpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 10 (effective, after forward-b

/Users/julian/miniforge3/envs/MI_pred/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function scale is deprecated; will be removed in 0.7.0. Use numpy.multiply instead.
  warnings.warn(msg, category=FutureWarning)


Fitted sphere radius:         96.2 mm
Origin head coordinates:      -0.8 14.4 43.1 mm
Origin device coordinates:    -0.8 14.4 43.1 mm
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 15 - 25 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 20 (effective, after forward-backward)
- Cutoffs at 15.00, 25.00 Hz: -6.02, -6.02 dB

Adding metadata with 4 columns
1 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1 events and 1024 original time points ...
0 bad epochs dropped
Adding metadata with 4 columns
1 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1 events and 1024 original time points ...
0 bad epochs dropped
Adding metadata with 4 columns
1 matching events found
No baseline correction applied
0 projection items activated
Using data

/Users/julian/miniforge3/envs/MI_pred/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function scale is deprecated; will be removed in 0.7.0. Use numpy.multiply instead.
  warnings.warn(msg, category=FutureWarning)


Elapsed: 3.168879270553589
Subject: 2
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

IIR filter parameters
---------------------
Butterworth highpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 10 (effective, after forward-backward)
- Cutoff at 1.00 Hz: -6.02 dB

Fitted sphere radius:         96.2 mm
Origin head coordinates:      -0.8 14.4 43.1 mm
Origin device coordinates:    -0.8 14.4 43.1 mm
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 15 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 20 (effective, after forward-backward)
- Cutoffs at 8.00, 15.00 Hz: -6.02, -6.02 dB

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

IIR filter parameters
---------------------
Butterworth highpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 10 

...

Elapsed: 3.2577033042907715
Subject: 52
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

IIR filter parameters
---------------------
Butterworth highpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 10 (effective, after forward-backward)
- Cutoff at 1.00 Hz: -6.02 dB

Fitted sphere radius:         96.2 mm
Origin head coordinates:      -0.8 14.4 43.1 mm
Origin device coordinates:    -0.8 14.4 43.1 mm
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 15 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 20 (effective, after forward-backward)
- Cutoffs at 8.00, 15.00 Hz: -6.02, -6.02 dB

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

IIR filter parameters
---------------------
Butterworth highpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 1

/Users/julian/miniforge3/envs/MI_pred/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function scale is deprecated; will be removed in 0.7.0. Use numpy.multiply instead.
  warnings.warn(msg, category=FutureWarning)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 15 - 25 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 20 (effective, after forward-backward)
- Cutoffs at 15.00, 25.00 Hz: -6.02, -6.02 dB

Adding metadata with 4 columns
1 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1 events and 1024 original time points ...
0 bad epochs dropped
Adding metadata with 4 columns
1 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1 events and 1024 original time points ...
0 bad epochs dropped
Adding metadata with 4 columns
1 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 1 events and 1024 original time points ...
0 bad epochs dropped
Adding metadata with 4 columns
1 matching even

/Users/julian/miniforge3/envs/MI_pred/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function scale is deprecated; will be removed in 0.7.0. Use numpy.multiply instead.
  warnings.warn(msg, category=FutureWarning)


Elapsed: 3.1277997493743896


In [12]:
Epochs_rest.keys()

dict_keys(['Subjects', 'Epochs_mts', 'Epochs_psd'])

## Training

### Load and organize data

In [84]:
load_path = '/Volumes/GoogleDrive/Mi unidad/PhD_Thesis/Data/GIGA/2Class/'
load_label_path = '/Volumes/GoogleDrive/Mi unidad/PhD_Thesis/Results/GIGA/2Class/'

In [85]:
data = pickle.load(open(load_path+'Epochs_rest.pkl', 'rb'))

In [86]:
data.keys()

dict_keys(['Subjects', 'Epochs_mts', 'Epochs_psd'])

In [87]:
X = []
for i in range(data['Epochs_mts'].shape[-1]):
    X+=features_by_channels(data['Epochs_mts'][:,:,:,i])[0:2] # for C3 and C4
X+=features_by_channels(data['Epochs_psd'])[0:2]# for C3 and C4

labels = pickle.load(open(load_label_path+'acc_train.pkl', 'rb'))

In [88]:
sc = scaler_list(StandardScaler())


In [98]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer
import copy

In [118]:
class MultiInputScaler(BaseEstimator, TransformerMixin):
    def __init__(self, scaler, input_feats, keep_dims=False):
        self.scaler = scaler
        self.input_feats = input_feats
        self.keep_dims = keep_dims
    
    def get_index(self):
        id = 0 
        idx = []
        for i in range(len(self.input_feats)):
            idx.append((id,id+self.input_feats[i]))
            id+=self.input_feats[i]
        return idx
    
    def feature_encoder(self):
        idx = self.get_index()
        return FunctionTransformer(
            func=lambda X: [X[:, i[0]:i[1]] for i in idx])

    def fit(self, X):
        X = self.feature_encoder().transform(X)
        self.scalers = [copy.deepcopy(self.scaler).fit(x) for x in X]
        return self

    def transform(self, X):
        X = self.feature_encoder().transform(X)
        Xt = [self.scalers[i].transform(X[i]) for i in range(len(X))]
        if self.keep_dims:
            Xt = np.concatenate(Xt, axis=-1)
        return Xt

In [129]:
sc2 = MultiInputScaler(scaler=StandardScaler(), input_feats=[x.shape[-1] for x in X], keep_dims=True)
X_tr = sc2.fit_transform(np.concatenate(X,axis=-1))

In [132]:
X_tr.shape

(49, 2106)

In [131]:
np.sum(X_tr-np.concatenate(X_train,axis=-1))

0.0

In [120]:
X_train

[array([[ 0.5320907 ,  0.21902125,  0.7375299 , ..., -0.47606173,
          0.46902898, -1.8813776 ],
        [ 0.05464523, -0.5609435 , -0.41022423, ..., -0.8543231 ,
          0.80247414, -0.5838277 ],
        [ 2.0536132 ,  3.1498287 ,  3.1076455 , ...,  0.72734   ,
         -0.6199859 ,  0.45146006],
        ...,
        [ 0.74552846, -0.62894607, -0.9810693 , ..., -0.4111354 ,
          0.34337494,  0.02103403],
        [-0.08983176, -0.6340523 , -1.0136843 , ..., -0.16637541,
          0.31000572, -1.0751611 ],
        [-0.031564  , -0.43589175, -0.64260715, ...,  0.425651  ,
         -0.288746  ,  0.9550342 ]], dtype=float32),
 array([[ 0.79533833,  1.5211245 ,  1.4672787 , ..., -0.20679194,
          0.2245391 ,  0.867201  ],
        [-1.3991458 , -0.4762429 , -0.2557136 , ..., -0.3879628 ,
          0.2664235 , -1.1627077 ],
        [-0.5848265 ,  1.5002843 ,  1.723514  , ...,  0.80261946,
         -0.727018  , -1.5825359 ],
        ...,
        [ 1.2201823 , -0.6878525 , -0.6

In [89]:
X_train = sc.fit_transform(X)

In [90]:
X_train[0].shape

(49, 270)

In [10]:
model_fun = DLSMR(input_feats = [x.shape[-1] for x in X_train])

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-08-11 22:59:11.593636: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-11 22:59:11.593746: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### 1 input scikeras

In [9]:
from sklearn.base import BaseEstimator, RegressorMixin
from scikeras.wrappers import KerasRegressor

In [17]:
from tensorflow.keras.initializers import GlorotNormal
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.models import Model



def DLSMR1(input_feats = 270, Q1f = 1.5, Qff = 0.5, l1 = 1e-3, l2 = 1e-3, act1 = 'tanh', actf = 'tanh', seed = 100):

    winitializer = GlorotNormal(seed=seed)
    binitializer = "zeros"
    
    inputs = Input(shape=input_feats, name='in_'+str(i))
    hiddens = Dense(input_feats*Q1f,activation=act1,kernel_regularizer=l1_l2(l1=l1,l2=l2),kernel_initializer=winitializer,bias_initializer=binitializer,name='h1_'+str(i))(inputs)
    
    #concat = concatenate(hiddens,name='concat')
    Qh = input_feats*Q1f
    
    hfi = Dense(Qh*Qff,activation=actf,kernel_regularizer=l1_l2(l1=l1,l2=l2),kernel_initializer=winitializer,bias_initializer=binitializer,name='hfi')(hiddens)
    
    output_c = Dense(1,activation="linear",kernel_initializer=winitializer, bias_initializer=binitializer,
                                 kernel_regularizer=l1_l2(l1=l1,l2=l2),name='outc')(hfi)
    
    return Model(inputs=inputs,outputs=[output_c])

In [24]:
m = KerasRegressor(model=DLSMR1,loss=tf.keras.losses.mae,
                    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                    epochs=200,batch_size=30,validation_split=0.1,
                    )

In [25]:
m.model().summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 in_1 (InputLayer)           [(None, 270)]             0         
                                                                 
 h1_1 (Dense)                (None, 405)               109755    
                                                                 
 hfi (Dense)                 (None, 202)               82012     
                                                                 
 outc (Dense)                (None, 1)                 203       
                                                                 
Total params: 191,970
Trainable params: 191,970
Non-trainable params: 0
_________________________________________________________________


In [26]:
m.fit(X=X_train[0],y=np.asarray(labels['3.0']))

Epoch 1/200
1/2 [==============>...............] - ETA: 0s - loss: 10.0789

2022-08-12 14:26:32.630055: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 1s 400ms/step - loss: 10.3059 - val_loss: 10.6821
Epoch 2/200
2/2 [==============================] - 0s 24ms/step - loss: 10.0752 - val_loss: 10.6884
Epoch 3/200
2/2 [==============================] - 0s 22ms/step - loss: 10.5763 - val_loss: 10.4949
Epoch 4/200
2/2 [==============================] - 0s 23ms/step - loss: 9.8095 - val_loss: 10.6482
Epoch 5/200
2/2 [==============================] - 0s 27ms/step - loss: 10.1585 - val_loss: 10.3244
Epoch 6/200
1/2 [==============>...............] - ETA: 0s - loss: 9.8212

2022-08-12 14:26:33.100149: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 0s 24ms/step - loss: 9.6681 - val_loss: 9.7949
Epoch 7/200
2/2 [==============================] - 0s 23ms/step - loss: 9.4898 - val_loss: 9.6932
Epoch 8/200
2/2 [==============================] - 0s 20ms/step - loss: 9.3898 - val_loss: 9.6830
Epoch 9/200
2/2 [==============================] - 0s 21ms/step - loss: 9.1278 - val_loss: 9.7487
Epoch 10/200
2/2 [==============================] - 0s 23ms/step - loss: 9.0935 - val_loss: 9.6136
Epoch 11/200
2/2 [==============================] - 0s 24ms/step - loss: 8.9415 - val_loss: 9.5595
Epoch 12/200
2/2 [==============================] - 0s 25ms/step - loss: 8.8424 - val_loss: 9.4949
Epoch 13/200
2/2 [==============================] - 0s 24ms/step - loss: 8.7345 - val_loss: 9.4100
Epoch 14/200
2/2 [==============================] - 0s 22ms/step - loss: 8.6500 - val_loss: 9.3267
Epoch 15/200
2/2 [==============================] - 0s 21ms/step - loss: 8.5126 - val_loss: 9.2773
Epoch 16/200
2/2 [========

KerasRegressor(
	model=<function DLSMR1 at 0x2d6719a60>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=<keras.optimizers.optimizer_v2.adam.Adam object at 0x2defb6d90>
	loss=<function mean_absolute_error at 0x2ad747040>
	metrics=None
	batch_size=30
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.1
	shuffle=True
	run_eagerly=False
	epochs=200
)

In [16]:
m.model_.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 in_1 (InputLayer)           [(None, 270)]             0         
                                                                 
 h1_1 (Dense)                (None, 270)               73170     
                                                                 
 hfi (Dense)                 (None, 135)               36585     
                                                                 
 outc (Dense)                (None, 1)                 136       
                                                                 
Total params: 109,891
Trainable params: 109,891
Non-trainable params: 0
_________________________________________________________________


In [14]:
model_fun = DLSMR1()

In [25]:
mdl = tf_wrapper(model=m1)#,loss=tf.keras.losses.mae,
              #optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3))

In [29]:
params = {'loss': [tf.keras.losses.mae, tf.keras.losses.mse]}
reg = GridSearchCV(mdl,param_grid=params,cv=3,scoring=r2_score)

In [30]:
reg.fit(X[0],np.asarray(labels['3.0']))

INFO:tensorflow:Assets written to: ram:///var/folders/q6/k111tjgn549dz55lkddwpryr0000gn/T/tmpfxp5m0nv/assets
INFO:tensorflow:Assets written to: ram:///var/folders/q6/k111tjgn549dz55lkddwpryr0000gn/T/tmptygao_al/assets


ValueError: Invalid parameter 'loss' for estimator tf_wrapper(model=<keras.engine.functional.Functional object at 0x29b5b1970>). Valid parameters are: ['model'].

In [15]:
model_fun.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 in_1 (InputLayer)           [(None, 270)]             0         
                                                                 
 h1_1 (Dense)                (None, 405)               109755    
                                                                 
 hfi (Dense)                 (None, 0)                 0         
                                                                 
 outc (Dense)                (None, 1)                 1         
                                                                 
Total params: 109,756
Trainable params: 109,756
Non-trainable params: 0
_________________________________________________________________


In [16]:
model_fun.compile(loss=tf.keras.losses.mae,
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3))
history = model_fun.fit(x=X_train[0], y=[np.asarray(labels['3.0'])],#Xz_psd, y=[zpca],#
                        epochs=200,batch_size=30,validation_split=0.1)

Epoch 1/200


2022-08-11 22:54:07.385429: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


: 

: 

### multiple input scikit-learn

In [91]:
from sklearn.base import BaseEstimator, RegressorMixin
from scikeras.wrappers import KerasRegressor
from sklearn.preprocessing import FunctionTransformer

In [92]:
class MultiInputRegressor(KerasRegressor):

    def get_index(self):
        id = 0 
        idx = []
        for i in range(len(self.input_feats)):
            idx.append((id,id+self.input_feats[i]))
            id+=self.input_feats[i]
        return idx
    
    @property
    def feature_encoder(self):
        idx = self.get_index()
        return FunctionTransformer(
            func=lambda X: [X[:, i[0]:i[1]] for i in idx],
        )

In [62]:
[270, 270*2, 270*3, 270*4, (270*4)+513, (270*4)+513]

[270, 540, 810, 1080, 1593, 1593]

In [93]:
Xc = np.concatenate(X_train,axis=-1)

In [76]:
Xc[:,1593:2106].shape

(49, 513)

In [61]:
a = [x.shape[-1] for x in X_train]
np.cumsum(a)

array([ 270,  540,  810, 1080, 1593, 2106])

In [80]:
idx = []
id = 0 
for i in range(len(a)):
    idx.append((id,id+a[i]))
    id+=a[i]

In [81]:
idx

[(0, 270), (270, 540), (540, 810), (810, 1080), (1080, 1593), (1593, 2106)]

In [55]:
X_train[4].shape

(49, 513)

In [94]:
m = MultiInputRegressor(model=DLSMR,loss=tf.keras.losses.mae,
                    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                    epochs=200,batch_size=30,validation_split=0.1,
                    input_feats = [x.shape[-1] for x in X_train]
                    )

In [95]:
m.fit(X=np.concatenate(X_train,axis=-1),y=np.asarray(labels['3.0']))

Epoch 1/200


2022-08-12 21:56:42.502715: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 1s 381ms/step - loss: 141.4291 - val_loss: 141.3406
Epoch 2/200
1/2 [==============>...............] - ETA: 0s - loss: 143.8177

2022-08-12 21:56:43.197640: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 0s 71ms/step - loss: 140.0397 - val_loss: 133.0301
Epoch 3/200
2/2 [==============================] - 0s 55ms/step - loss: 130.9276 - val_loss: 128.6880
Epoch 4/200
2/2 [==============================] - 0s 65ms/step - loss: 128.4677 - val_loss: 123.1001
Epoch 5/200
2/2 [==============================] - 0s 45ms/step - loss: 121.2383 - val_loss: 117.3014
Epoch 6/200
2/2 [==============================] - 0s 52ms/step - loss: 117.1386 - val_loss: 113.1984
Epoch 7/200
2/2 [==============================] - 0s 50ms/step - loss: 112.6957 - val_loss: 107.4946
Epoch 8/200
2/2 [==============================] - 0s 113ms/step - loss: 107.2777 - val_loss: 104.8345
Epoch 9/200
2/2 [==============================] - 0s 53ms/step - loss: 104.3658 - val_loss: 100.5124
Epoch 10/200
2/2 [==============================] - 0s 51ms/step - loss: 100.0318 - val_loss: 96.3482
Epoch 11/200
2/2 [==============================] - 0s 53ms/step - loss: 95.8513 - val_loss: 

MultiInputRegressor(
	model=<function DLSMR at 0x29e816e50>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=<keras.optimizers.optimizer_v2.adam.Adam object at 0x13c5312e0>
	loss=<function mean_absolute_error at 0x2ad747040>
	metrics=None
	batch_size=30
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.1
	shuffle=True
	run_eagerly=False
	epochs=200
	input_feats=[270, 270, 270, 270, 513, 513]
)

In [39]:
np.concatenate(X_train,axis=-1).shape

(49, 2106)

### inference

In [54]:
ypred = model_fun.predict(X_train)

2/2 [==============================] - 0s 14ms/step


2022-08-11 17:53:47.095832: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [27]:
from sklearn.metrics import r2_score

In [56]:
r2_score(ypred.ravel(),np.asarray(labels['3.0']))

0.7757552388388395

In [57]:
np.asarray(labels['3.0'])

array([0.715     , 0.51      , 0.815     , 0.595     , 0.72      ,
       0.775     , 0.48333333, 0.495     , 0.53333333, 0.59      ,
       0.515     , 0.54      , 0.635     , 0.87      , 0.485     ,
       0.535     , 0.515     , 0.53      , 0.59      , 0.53      ,
       0.695     , 0.52      , 0.685     , 0.49      , 0.54      ,
       0.62      , 0.56      , 0.585     , 0.545     , 0.52      ,
       0.615     , 0.525     , 0.55      , 0.81      , 0.515     ,
       0.7       , 0.55      , 0.58      , 0.51      , 0.545     ,
       0.555     , 0.955     , 0.645     , 0.57      , 0.555     ,
       0.68      , 0.845     , 0.51      , 0.54      ])